## Comparing RTDP with conventional DP methods



In [24]:
import numpy as np
import matplotlib.pyplot as plt

### Environment - Taxi Domain
In the taxi domain, an agent navigates on a 5x5 grid with immovable objects in episodic manner. The taxi agent has to transport passengers from and to some locations predefined by the environment in this girdworld - Red, Green, Yellow and Blue. In each episode, taxi starts in a random location with a passenger waiting randomlyat one of these four locations and wishing to be dropped-off at one of the four locations chosen randomly. The taxi agent can only move in one of the four cardinal directions, pickup the passenger when in same location as the passenger and drop off the passenger. Every episode ends with the passenger dropped off at the desired location.
 
There is a reward of -1 for every timestep. Aim is to seek a policy which maximises the total reward at the end of an episode. There are 500 different possible states the agent can be in - 25 locations on grid, 5 location of passenger and 4 drop-off spots.

In [25]:
RED = 0
GREEN = 1
YELLOW = 2
BLUE = 3
CAR = 4

location = {RED:0, GREEN:4, YELLOW:20, BLUE:23}


trans = np.array([[0,5,0,1],
              [1,6,0,1],
              [2,7,2,3],
              [3,8,2,4],
              [4,9,3,4],
              [0,10,5,6],
              [1,11,5,6],
              [2,12,7,8],
              [3,13,7,9],
              [4,14,8,9],
              [5,15,10,11],
              [6,16,10,12],
              [7,17,11,13],
              [8,18,12,14],
              [9,19,13,14],
              [10,20,15,15],
              [11,21,16,17],
              [12,22,16,17],
              [13,23,18,19],
              [14,24,18,19],
              [15,20,20,20],
              [16,21,21,22],
              [17,22,21,22],
              [18,23,23,24],
              [19,24,23,24]]).astype("int8")


class Taxi(object):
    class observations(object):
        def __init__(self):
            self.car = np.random.randint(25)
            self.passenger = np.random.choice([RED, GREEN, YELLOW, BLUE, CAR])
            self.destination = np.random.choice([RED, GREEN, YELLOW, BLUE])
        
        def reset(self):
            self.car = np.random.randint(25)
            self.passenger = np.random.choice([RED, GREEN, YELLOW, BLUE, CAR])
            self.destination = np.random.choice([RED, GREEN, YELLOW, BLUE])
            return self.getFeature()    
        
        def getState(self):
            return self.car, self.passenger, self.destination
        
        def getFeature(self):
            return self.destination + self.passenger*4 + self.car*5*4
            
        def setState(self, feature):
            self.car = feature/20
            feature %= 20
            self.passenger = feature/4
            feature %= 4
            self.destination = feature 

        def sample(self):
            return self.reset()

    def __init__(self):
        self.trans = trans
        self.observation_space = self.observations()
        self.acc_reward = 0
        self.done = False
        self.reset()

    def reset(self):
        self.acc_reward = 0
        self.done = False
        return self.observation_space.reset()

    def totalReward(self):
        return self.acc_reward

    def goalState(self):
        if self.observation_space.passenger == CAR:
            return location[self.observation_space.destination]
        else:
            return location[self.observation_space.passenger]

    def step(self, action):
        reward = -1
        if action<4:
            self.observation_space.car = self.trans[self.observation_space.car,action]
            """if self.picked:
                self.observation_space.passenger = self.observation_space.car"""
        elif action == 4:
            if self.observation_space.passenger!=CAR and self.observation_space.car == location[self.observation_space.passenger]:
                reward = 0
                self.observation_space.passenger = CAR
        elif action == 5:
            if self.observation_space.car==location[self.observation_space.destination] and self.observation_space.passenger==CAR:
                reward = 1
                self.done = True
        self.acc_reward += reward
        return self.observation_space.getFeature(), reward, self.done

    def episodeEnded(self):
        return self.done


In [26]:
# learning reward function
demoT = Taxi()
r = np.zeros((500,6))
t = np.zeros((500,6))

for state in range(500):
    for action in range(6):
        demoT.observation_space.setState(state)
        t[state, action], r[state, action], _ = demoT.step(action)

r[s,a] = Reward received when action 'a' is performed in state s

t[s,a] = Taxi transitions to this state after excution of action a in state s

In [10]:
# constants
gamma = 0.85

Utility Functions -
to check convergence to optimal policy. As the MDP defined above can experience vastly different costs for starting in different states, average cost on a small number of random restarts is not a good way to test convergence to optimal policy. As a work around this problem, I'll be comparing the costs to those of certain fixed starting states with a greedy policy over V*. (I use stationary, $\delta \lt 10^{-7}$, approximate Value Function from Value Iteration) 

In [80]:
vStar = None
testingStates = None

def idealRewardFromRun(env, states):
    global vStar
    if vStar==None:
        vStar=VI()
    idealRewards={}

    for st in states:
        env.reset()
        env.observation_space.setState(st)
        
        s = st
        done = False
        while not done:
            q=np.zeros(6)
            for a in range(6):
                q[a] = (r[s,a]+gamma*vStar[t[s,a]])
            action =np.argmax(q)
            s,_, done = env.step(action)
        idealRewards[st] = env.totalReward()
    return idealRewards 

def testConvergence(env, valueFunc, margin=3):
    global testingStates
    if testingStates==None:
        testingStates = idealRewardFromRun(env, [158])
        print "testingStates generated!"
        
    for st in testingStates.keys():
        epoch=0
        env.reset()
        env.observation_space.setState(st)
        lim = int((abs(testingStates[st])+margin)*1.1)
        s=st
        done=False
        while (not done) and epoch<lim:
            epoch+=1
            val = [(a, r[s,a]+gamma*valueFunc[t[s,a]]) for a in range(6)]
            np.random.shuffle(val)
            action = max(val, key=lambda pair:pair[1])[0]
            s,_,done = env.step(action)
        if epoch==lim:
            return False
    return True

def VI(eps = 0.0000001):
    v = np.zeros(500)
    delta = np.inf
    sweep = 0
    
    while(delta>eps and sweep<1000):
        sweep +=1
        delta=0
        v_old = np.copy(v)

        for state in range(500):
            temp=[]
            for action in range(6):
                temp.append(r[state, action] + gamma*v_old[t[state, action]])
                
            v[state] = max(temp)
            delta = max(delta, abs(v[state] - v_old[state]))
    return v

In [36]:
# testing Convergence utilities -should't converge using a random policy
print idealRewardFromRun(Taxi(),np.random.randint(0,500, 10))
print testConvergence(Taxi(), np.random.random(500))

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:61: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:19: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


{320: -3, 385: -2, 261: -12, 262: -8, 70: -6, 272: -8, 19: -6, 358: -4, 92: -11, 29: -12}
testingStates generated!
False


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:6: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:40: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


## Synchronous dynamic programming 
I use Value Iteration to show results.
We can think about this case fo DP as one where updates to all the states are applied at the same time. Therefore, an update for $V_{k+1}$ uses estimates $V_k$.

In [37]:
def ValueIteration(eps = 0.0000001):
    v = np.zeros(500)
    delta = np.inf
    converged = False
    sweep = 0
    updates_opt_policy = 0
    updates = 0
    while(delta>eps and sweep<1000):
        sweep +=1
        delta=0
        v_old = np.copy(v)

        for state in range(500):
            updates+=1
            temp=[]
            for action in range(6):
                temp.append(r[state, action] + gamma*v_old[t[state, action]])
                
            v[state] = max(temp)
            delta = max(delta, abs(v[state] - v_old[state]))
            if not converged:
                converged = testConvergence(Taxi(), v)
                updates_opt_policy = updates
    return v, sweep, updates, updates_opt_policy

### Gauss-Seidel dynamic programming
It is a type of Asyncronous DP. Gauss-Seidel DP differs from the synchronous version in that the costs are backed
up one state at a time in a sequential “sweep” of all the states, with the computation for each state using the most recent costs of the other states. If we assume that the states are numbered in order, as we have here, and that each sweep proceeds in this order, then the result of each iteration of Gauss-Seidel DP can be written as follows: 

$ \forall$ states $s_i$ and k = 0, 1, ... 

$ V_{k+1}(s_i)$ = $max_a$ [ r($s_i$,a) + $\gamma$V($s_j$) ];

where $V(s_j)$ = $ V_{k+1}(s_j) $ if j$\lt$i, else $ V_{k}(s_j) $


In [48]:
def GaussSeidelValueIteration(eps=0.0000001):
    v = np.zeros(500)
    delta = np.inf
    sweep = 0
    updates = 0
    updates_opt_policy = 0
    converged = False
    
    while(delta>eps and sweep<1000):
        sweep +=1
        delta=0
        v_old = np.copy(v)

        for state in range(500):
            temp=[]
            for action in range(6):
                temp.append(r[state, action] + gamma*v[t[state, action]])
            v[state] = max(temp)
            updates+=1
            
            delta = max(delta, abs(v[state] - v_old[state]))
        if not converged:
            converged = testConvergence(Taxi(), v)
            updates_opt_policy = updates   
    return v, sweep, updates, updates_opt_policy


### Real-Time Dynamic Programming
The Gauss-Seidel DP algorithm implement above is off-line for solving Markovian decision problems. Although they successively approximate the optimal evaluation function through a sequence of stages, these stages are not related to the time steps of the decision problem being solved. RLDP we consider here performs asynchronous DP concurrently with the actual process of control. 

RTDP refers to cases in which the concurrently executing DP and controller influence one another as follows:

First, the controller always follows a greedy policy w.r.t. the most recent estimate of $V^{*}$. This means that $a_t$ is always the greedy action with respect to $V_{k_t}$. Any ties in selecting these actions is resolved randomly, which ensures the continuing selection of all the greedy actions. 

Second, between the execution of $a_t$, and $a_{t+1}$, the cost of s, is always backed up, but more generally, any state can also be updates in this trial. 

In [102]:
def RLDP(env, eps=0.0000001):
    v = 0.5*np.zeros(500)
    epoch = 0
    converged = False
    updates=0
    while(not converged and epoch<1000):
        epoch +=1
        s = env.reset()
        done = False
        while not done:
            q=np.zeros(6)
            for a in range(6):
                q[a] = (r[s,a]+gamma*v[t[s,a]])
            action =np.argmax(q)
            v[s] = np.max(q)
            updates+=1
            s, _, done = env.step(action)
        
        converged = testConvergence(env, v)
    
    return v, epoch, updates

_, epochs, totalUpdates = RLDP(Taxi())
print "RLDP"
print "converged within", epochs,"epochs with just",totalUpdates,"updates."

 RLDP
converged within 6 epochs with just 339 updates.


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:13: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:40: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [100]:
print "Value Iteration"
_, sweeps, totalUpdates, UpdatesForConvergence = ValueIteration()
print "converged to optimal values in",sweeps,"sweeps with total",totalUpdates
print "Convergence for test starting states was reached with",UpdatesForConvergence,"updates"

Value Iteration


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:17: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:40: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


converged to optimal values in 101 sweeps with total 50500
Convergence for test starting states was reached with 1046 updates


In [92]:
print "Gauss-Seidel Value Iteration"
_, sweeps, totalUpdates, UpdatesForConvergence = GaussSeidelValueIteration()
print "converged to optimal values in",sweeps,"sweeps with total",totalUpdates
print "Convergence for test starting states was reached with",UpdatesForConvergence,"updates"

Gauss-Seidel Value Iteration


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:17: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:40: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


converged to optimal values in 101 sweeps with total 50500
Convergence for test starting states was reached with 2000 updates


As can be seen, RLDP converges to optimal values for relevant states much fast (lesser totat updates for cenvergence) compared to classical DP methods.